In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import Counter

In [2]:
def remove_rt(text):
    return re.sub('<rt>.+?</rt>', '', text)

def tag(text):
    text = re.sub(r'<span class="colorC">(.+?)</span>', r"{org}\1{/org}", text)
    text = re.sub(r'<span class="colorL">(.+?)</span>', r"{plc}\1{/plc}", text)
    text = re.sub(r'<span class="colorN">(.+?)</span>', r"{per}\1{/per}", text)
    return text

def retag(text):
    text = re.sub(r'{org}(.+?){/org}', r"<org>\1</org>", text)
    text = re.sub(r'{plc}(.+?){/plc}', r"<plc>\1</plc>", text)
    text = re.sub(r'{per}(.+?){/per}', r"<per>\1</per>", text)
    return text

def remove_a(text):
    text = re.sub(r'</?a.*?>', '', text)
    text = re.sub(r'<span class="under">(\w+)</span>', r'\1', text)
    text = re.sub(r'<img.+?>(<br ?/?>)?', '', text)
    text = re.sub(r'^<br ?/?>', '', text)
    return text.strip()

# for old web easy
def scrape_easy_one(html):
    soup = BeautifulSoup(html, "html.parser")
    url_normal = soup.find('div', id="regularnews")
    if url_normal == None:
        url_normal = soup.find('p', id="regularnews")
    url_normal = url_normal.a.get('href')
    if '/http://' in url_normal:
        url_normal = 'http://' + url_normal.split('/http://')[-1]
    else:
        url_normal = 'https://' + url_normal.split('/https://')[-1]
    date = soup.find('p', id="newsDate").text[1:-1]
    #title_easy = soup.find('h1', class_="article-main__title")
    #title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    url_easy = soup.find('meta', attrs={'name':'shorturl'}).get('content')
    title_easy = soup.find('div', id='newstitle').h2
    title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    title_easy = BeautifulSoup(remove_rt(str(title_easy)), "html.parser").text.strip()
    article_easy = soup.find('div', id="newsarticle")
    article_easy = BeautifulSoup(tag(remove_rt(str(article_easy))), "html.parser").text.strip()
    article_easy_ruby = soup.find('div', id="newsarticle").find_all('p')
    article_easy_ruby = '\n'.join([''.join([remove_a(str(l)) for l in p.contents]) for p in article_easy_ruby if p != []]).strip()
    
    return {
        'id':url_easy.split('/')[-1].split('.html')[0],
        'title_easy':title_easy,
        'title_easy_ruby':title_easy_ruby,
        'article_easy':retag(article_easy),
        'article_easy_ruby':article_easy_ruby,
        'url_easy':url_easy,
        'url_normal':url_normal,
        'date_easy':date
    }

# for new web easy
def scrape_easy_one_new(html):
    soup = BeautifulSoup(html, "html.parser")
    url_easy = 'https://' + soup.find('meta', property="og:url").get('content').split('/https://')[-1]
    url_normal = soup.find('div', class_="link-to-normal").a.get('href')
    date = soup.find('p', class_="article-main__date").text[1:-1]
    title_easy = soup.find('h1', class_="article-main__title")
    title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    title_easy = BeautifulSoup(remove_rt(str(title_easy)), "html.parser").text.strip()
    article_easy = soup.find('div', class_="article-main__body article-body")
    article_easy = BeautifulSoup(tag(remove_rt(str(article_easy))), "html.parser").text.strip()
    article_easy_ruby = soup.find('div', class_="article-main__body article-body").find_all('p')
    article_easy_ruby = '\n'.join([''.join([remove_a(str(l)) for l in p.contents]) for p in article_easy_ruby if p != []]).strip()
    
    return {
        'id':url_easy.split('/')[-1].split('.html')[0],
        'title_easy':title_easy,
        'title_easy_ruby':title_easy_ruby,
        'article_easy':retag(article_easy),
        'article_easy_ruby':article_easy_ruby,
        'url_easy':url_easy,
        'url_normal':url_normal,
        'date_easy':date
    }



def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    with open('nolinknormal.txt') as f:
        nolink = f.read().split()
    return sorted(set(notyet) - set(nolink))
        
def js_e(dic):
    with open('nhkwebeasy.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkwebeasy.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)

# scrape

In [4]:
driver = webdriver.Chrome()

# clean category & keyword

In [4]:
with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
    print(len(data))
with open('nhkwebeasy.json','r', encoding='utf-8') as f:
    data = json.load(f)
    print(len(data))

8398
4744


In [47]:
# check category

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print('articles: ', len(data))
genre = Counter()
for dic in data:
    for g in dic['genre']:
        genre[g] += 1
genre.most_common()

articles:  8730


[('社会', 2617),
 ('国際', 2336),
 ('科学・文化', 1443),
 ('ビジネス', 1345),
 ('スポーツ', 1071),
 ('政治', 772),
 ('暮らし', 732),
 ('地域', 615),
 ('気象・災害', 414)]

In [46]:
# genre <> keywords

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)

category = ['社会', '国際', 'ビジネス', 'スポーツ', '政治', '科学・文化', '暮らし', '地域', '気象・災害']
for i, dic in enumerate(data):
    newgenre = []
    newkey = []
    for j in dic['genre']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        elif j == "科学・医療" or j == "文化・エンタメ":
            newgenre.append('科学・文化')
        elif j == "経済":
            newgenre.append('ビジネス')
        else:
            newkey.append(j)
    for j in dic['keywords']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        else:
            newkey.append(j)
    data[i]['genre'] = list(set(newgenre))
    data[i]['keywords'] = list(set(newkey))

with open('nhkweb.json','w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
    

In [3]:
normal = pd.read_json('nhkwebeasy.json')
normal.id.value_counts()

k10010353911000    1
k10010441541000    1
k10012115201000    1
k10010055061000    1
k10010471031000    1
                  ..
k10010771931000    1
k10010399941000    1
k10010904951000    1
k10011871991000    1
k10011099221000    1
Name: id, Length: 4530, dtype: int64

# NHK web easy

In [18]:
with open('linkeasy.txt') as f:
    urls = f.read().strip().split()
    ids = set([x.split('.html')[0].split('/')[-1] for x in urls])

id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())
print(len(ids), len(id_exist))
len(ids - id_exist)

918 4850


892

In [4]:
driver = webdriver.Chrome()

In [19]:
id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())
for url in urls:
    ID = url.split('.html')[0].split('/')[-1]
    if ID in id_exist:
        continue
    driver.get(url)
    time.sleep(6)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time|times).*?<a href="(.+?)">', html)
    if snap == None:
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(10)
    html = driver.page_source.encode('utf-8')
    try:
        dic = scrape_easy_one(html)
    except:
        dic = scrape_easy_one_new(html)
    js_e(dic)
    id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())

AttributeError: 'NoneType' object has no attribute 'get'

In [41]:
html = driver.page_source.encode('utf-8')
dic = scrape_easy_one(html)
dic

{'id': 'k10014846591000',
 'title_easy': 'ウナギの餌７５％はアメリカザリガニ',
 'title_easy_ruby': 'ウナギの<ruby>餌<rt>えさ</rt></ruby>７５％はアメリカザリガニ',
 'article_easy': '天然のウナギは捕れる量がとても少なくなっています。<org>東京大学</org>の研究チームは、平成２０年（２００８年）と２１年（２００９年）、<plc>岡山県</plc>の<plc>旭川</plc>で季節や場所を変えて捕まえた５０匹のウナギの体の中に残っていた餌を調べました。そして、餌の７４．９％が外国から入って来たアメリカザリガニだったことが分かりました。ウナギが何をよく食べているか、一つの川で詳しく調べたのは初めてです。研究チームは、川にいる生き物や環境が変わったために、ウナギが少なくなっていると考えています。また、餌はカワエビなど<plc>日本</plc>のものでなく、外国から入って来た生き物が多くなっている可能性があると話しています。<org>東京大学</org>で研究している<per>海部</per><per>健三</per>さんは「ウナギが今より少なくならないように、全国の川でも同じように調べる必要があります」と話しています。',
 'article_easy_ruby': '<ruby>天然<rt>てんねん</rt></ruby>のウナギは<ruby>捕<rt>と</rt></ruby>れる<ruby>量<rt>りょう</rt></ruby>がとても<ruby>少<rt>すく</rt></ruby>なくなっています。\n\n<span class="colorC"><ruby>東京大学<rt>とうきょうだいがく</rt></ruby></span>の<ruby>研究<rt>けんきゅう</rt></ruby>チームは、<ruby>平成<rt>へいせい</rt></ruby>２０<ruby>年<rt>ねん</rt></ruby>（２００８<ruby>年<rt>ねん</rt></ruby>）と２１<ruby>年<rt>ねん</rt></ruby>（２００９<ruby>年<rt>ねん</rt></ruby>）、<span class="colorL"><ruby>岡山県<r

In [42]:
js_e(dic)

In [30]:
html

 #social_b15 a { position: absolute; top: 20px; left: undefinedpx; } --></style><li id="social_sns_notice" style="clear:both;"><img src="https://web.archive.org/web/20120406124107/http://www9.nhk.or.jp/common/social/img/sns_notice.gif" style="border-width: 0px;"></li></ul></div></div></div>\n    <div id="subMenu"></div>\n    </div>\n  </div>\n  <div id="content">\n       <div class="wrap png_bg">\n      <h1><a href="index.html"><img src="/web/20120406065233im_/http://www3.nhk.or.jp/news/easy/img/hd_title.png" alt="NEWS WEB EASY"></a></h1>\n      <div id="main">\n        <!--div id="today">\n          <p><img src="img/img_today.png" alt="\xe4\xbb\x8a\xe6\x97\xa5\xe3\x81\xaf\xe2\x80\xa6" />\xe3\x80\x802012<ruby><rb>\xe5\xb9\xb4</rb><rp>(</rp><rt>\xe3\x81\xad\xe3\x82\x93</rt><rp>)</rp></ruby>1<ruby><rb>\xe6\x9c\x88</rb><rp>(</rp><rt>\xe3\x81\x8c\xe3\x81\xa4</rt><rp>)</rp></ruby>11<ruby><rb>\xe6\x97\xa5</rb><rp>(</rp><rt>\xe3\x81\xab\xe3\x81\xa1</rt><rp>)</rp></ruby> <ruby><rb>\xe6\xb0\xb4